**강사: 멀티캠퍼스 강선구 (sunku0316.kang@multicampus.com)**

**ProDS 시험장 환경**

||version|
|-|-----|
|python|3.7.4|
|numpy|1.18.5|
|pandas|0.25.1|
|sklearn|0.21.3|
|scipy|1.5.2|

In [51]:
import numpy as np
import pandas as pd
import sklearn
import scipy

import sys
print(sys.version)

for i in [np, pd, sklearn, scipy]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
numpy 1.18.5
pandas 0.25.1
sklearn 0.21.3
scipy 1.5.2


# 실습 문제 1

데이터셋: **Dataset_07.csv**, comma(",") 400rows, 9columns, UTF-8 인코딩

|컬럼|정의|Type|
|----|----|----|
|Serial_No|구분자 번호|Double|
|GRE| GRE 시험성적|Double|
|TOEFL| TOEFL 시험성적|Double|
|University_Rating| 대학 평가 그룹(1~5)|Double|
|SOP|자기 소개서 점수(1~5)|Double|
|LOR|추천서 점수(1~5)|Double|
|CGPA|학부 평량 평점(10점 만점 환산 점수)|Double|
|Research| 연구 참여 경험 여부(0/1)|Double|
|Chance_of Admit|합격 가능성|Double|

In [52]:
df_07 = pd.read_csv('Dataset_07.csv', index_col='Serial_No')
df_07.head()

,GRE,TOEFL,University_Rating,SOP,LOR,CGPA,Research,Chance_of_Admit
Serial_No,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,324,107,4,4.0,4.5,8.87,1,0.76
3,316,104,3,3.0,3.5,8.00,1,0.72
4,322,110,3,3.5,2.5,8.67,1,0.80
5,314,103,2,2.0,3.0,8.21,0,0.65


In [53]:
df_07.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 1 to 400
Data columns (total 8 columns):
GRE                  400 non-null int64
TOEFL                400 non-null int64
University_Rating    400 non-null int64
SOP                  400 non-null float64
LOR                  400 non-null float64
CGPA                 400 non-null float64
Research             400 non-null int64
Chance_of_Admit      400 non-null float64
dtypes: float64(4), int64(4)
memory usage: 28.1 KB


In [54]:
df_07.describe()

,GRE,TOEFL,University_Rating,SOP,LOR,CGPA,Research,Chance_of_Admit
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,316.807500,107.410000,3.087500,3.400000,3.452500,8.598925,0.547500,0.724350
std,11.473646,6.069514,1.143728,1.006869,0.898478,0.596317,0.498362,0.142609
min,290.000000,92.000000,1.000000,1.000000,1.000000,6.800000,0.000000,0.340000
25%,308.000000,103.000000,2.000000,2.500000,3.000000,8.170000,0.000000,0.640000
50%,317.000000,107.000000,3.000000,3.500000,3.500000,8.610000,1.000000,0.730000
75%,325.000000,112.000000,4.000000,4.000000,4.000000,9.062500,1.000000,0.830000
max,340.000000,120.000000,5.000000,5.000000,5.000000,9.920000,1.000000,0.970000


**1**. 합격 가능성에 GRE, TOEFL, CGPA 점수 가운데 가장 영향이 큰 것이 어떤 점수인지 알아 보기 위해서 상관 분석을 수행한다. 

- 피어슨(Pearson) 상관계수 값을 구한다.
- Chance_of_Admit와의 가장 큰 상관계수 값을 가지는 항목의 상관계수를 소수점 넷째 자리에서 반올림하여 셋째 자리까지 기술하시오. 


(답안 예시) 0.123


In [55]:
X = ['Chance_of_Admit', 'GRE', 'TOEFL', 'CGPA']
df_07[X].corr().loc['Chance_of_Admit'].iloc[1:].max()

0.8732890993553011

**0.873**

**2**. GRE 점수의 평균 이상을 받은 그룹과 평균 미만을 받은 그룹의 CGPA 평균은 차이가 있는지 검정을 하고자 한다.

- 적절한 검정 방법을 선택하고 양측 검정을 수행하시오 (등분산으로 가정)

- 검정 결과, 검정통계량의 추정치를 소수점 셋째 자리에서 반올림하여 소수점 두 자리까지 기술하시오.


Python 함수 가이드
```
scipy.stats에 있는 기능 활용
```

In [56]:
from scipy.stats import ttest_ind # dir메소트

m = df_07['GRE'].mean()
ttest_ind(
    df_07.loc[df_07['GRE'] >= m, 'CGPA'],
    df_07.loc[df_07['GRE'] < m, 'CGPA'],
    equal_var=True
)

Ttest_indResult(statistic=19.443291692470982, pvalue=1.1127037052806161e-59)

**19.44**

**3**. Chance_of_Admit 확률이 0.5를 초과하면 합격으로, 이하이면 불합격으로 구분하고 로지스틱 회귀분석을 수행하시오.

- 원데이터만 사용하고, 원데이터 가운데 Serial_No와 Label은 모형에서 제외
- 각 설정값은 다음과 같이 지정하고, 언급되지 않은 사항은 기본 설정값을 사용하시오Seed : 123
- 로지스틱 회귀분석 수행 결과에서 로지스틱 회귀계수의 절대값이 가장 큰 변수와 그 값을 기술하시오. (로지스틱 회귀계수는 반올림하여 소수점 둘째 자리까지 / Intercept는 제외)

(답안 예시) abc, 0.12


Python 함수 가이드
```
sklearn.linear_model.LogisticRegression, solver='liblinear', random_state=123
```

In [57]:
from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression(solver='liblinear', random_state=123)
X = df_07.columns[:-1]
clf_lr.fit(df_07[X], df_07['Chance_of_Admit'] > 0.5)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [58]:
clf_lr.coef_

array([[-0.08896188,  0.10548837,  0.05410951, -0.26490356,  1.01891822,
         2.06269373,  0.63495759]])

**2.06**

# 실습 문제 2

데이터셋: **Dataset_06.csv**, comma(",") 4323rows, 19columns, UTF-8 인코딩

|id|매물 번호|Double|
|--|---------|------|
|date|날짜|String|
|price|거래 가격|Double|
|bedrooms|방 개수|Double|
|bathrooms|화장실 개수 (화장실은 있으나 샤워기 없는 경우 0.5로 처리)|Double|
|sqft_living|건축물 면적|Double|
|sqft_lot|대지 면적|Double|
|floors|건축물의 층수|Double|
|waterfront|강변 조망 가능 여부 (0 / 1)|Double|
|view|경관 (나쁨에서 좋음으로 0 ~ 4로 표시)|Double|
|condition|관리 상태 (나쁨에서 좋음으로 1 ~ 5로 표시)|Double|
|grade|등급 (낮음에서 높음으로 1 ~ 13으로 표시)|Double|
|sqft_above|지상 면적|Double|
|sqft_basement|지하실 면적|Double|
|yr_built|건축 연도|Double|
|yr_renovated|개축 연도|Double|
|zipcode|우편번호|Double|
|sqft_living15|15개의 인근 주택의 평균 건물 면적|Double|
|sqft_lot15	|15개의 인근 주택의 평균 대지 면적|Double|


In [59]:
df_06 = pd.read_csv('Dataset_06.csv')
df_06.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,sqft_living15,sqft_lot15
0,2597000006,20150309T000000,347500.0,3,1.50,1180,8353,1.0,0,0,3,7,1180,0,1960,0,98155,1710,8748
1,8813400345,20150414T000000,575000.0,2,1.00,980,3663,1.0,0,0,5,7,980,0,1909,0,98105,1620,3706
2,226059106,20150102T000000,489500.0,3,1.75,2090,65558,1.0,0,0,3,8,1330,760,1977,0,98072,2450,47178
3,8645500270,20141028T000000,246000.0,4,1.75,1720,7455,1.0,0,0,4,7,1020,700,1963,0,98058,1720,7700
4,7010700550,20141114T000000,595000.0,3,2.50,2030,5100,2.0,0,0,3,7,2030,0,2008,0,98199,1790,4380


In [60]:
df_06.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4323 entries, 0 to 4322
Data columns (total 19 columns):
id               4323 non-null int64
date             4323 non-null object
price            4323 non-null float64
bedrooms         4323 non-null int64
bathrooms        4323 non-null float64
sqft_living      4323 non-null int64
sqft_lot         4323 non-null int64
floors           4323 non-null float64
waterfront       4323 non-null int64
view             4323 non-null int64
condition        4323 non-null int64
grade            4323 non-null int64
sqft_above       4323 non-null int64
sqft_basement    4323 non-null int64
yr_built         4323 non-null int64
yr_renovated     4323 non-null int64
zipcode          4323 non-null int64
sqft_living15    4323 non-null int64
sqft_lot15       4323 non-null int64
dtypes: float64(3), int64(15), object(1)
memory usage: 641.8+ KB


In [61]:
df_06.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,sqft_living15,sqft_lot15
count,4.323000e+03,4.323000e+03,4323.000000,4323.000000,4323.000000,4.323000e+03,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000
mean,4.611898e+09,5.410756e+05,3.394633,2.110282,2096.345362,1.617258e+04,1.488434,0.007634,0.229933,3.405274,7.669443,1797.350914,298.994448,1970.928985,83.121906,98079.048346,2001.428406,13510.676613
std,2.852472e+09,3.588669e+05,1.026123,0.777992,923.456345,5.076933e+04,0.531715,0.087046,0.766526,0.649985,1.184177,832.287622,445.927769,29.409668,398.840275,54.020462,692.858113,33426.068810
min,3.800008e+06,7.500000e+04,0.000000,0.000000,384.000000,6.380000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,384.000000,0.000000,1900.000000,0.000000,98001.000000,620.000000,660.000000
25%,2.131451e+09,3.220000e+05,3.000000,1.500000,1430.000000,5.065500e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1200.000000,0.000000,1951.000000,0.000000,98033.000000,1500.000000,5120.000000
50%,3.914000e+09,4.500000e+05,3.000000,2.250000,1930.000000,7.644000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1570.000000,0.000000,1974.000000,0.000000,98070.000000,1850.000000,7653.000000
75%,7.334501e+09,6.444750e+05,4.000000,2.500000,2580.000000,1.074400e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2230.000000,580.000000,1997.000000,0.000000,98118.000000,2370.000000,10108.000000
max,9.842300e+09,3.635000e+06,33.000000,6.500000,7850.000000,1.074218e+06,3.500000,1.000000,4.000000,5.000000,13.000000,7850.000000,2620.000000,2015.000000,2015.000000,98199.000000,6210.000000,871200.000000


**1** 강변 조망이 가능한지 여부(waterfront)에 따라 평균 주택 가격을 계산하고 

조망이 가능한 경우와 그렇지 않은 경우의 평균 가격 차이의 절대값을 구하시오. 

답은 소수점 이하는 버리고 정수부만 기술하시오. 

(답안 예시) 1234567


In [62]:
a, b = df_06.groupby('waterfront')['price'].mean()
a, b, b-a

(532165.1307692308, 1699437.878787879, 1167272.748018648)

**1167272**

**2** price, bedrooms, bathrooms, sqft_living, sqft_lot, floors, yr_built 등 7개의 변수 간의 상관분석을 수행하고 

price와의 상관계수의 절대값이 가장 큰 변수와 가장 작은 변수를 차례로 기술하시오. 

(답안 예시) view, zipcode


In [63]:
X = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'yr_built']
df_06[X].corr().loc['price'].abs().iloc[1:].sort_values().iloc[[-1, 0]].index

Index(['sqft_living', 'yr_built'], dtype='object')

**sqft_living', 'yr_built**

**3** id, date, 그리고 zipcode를 제외한 모든 변수를 독립변수로, price를 종속변수로 하여 회귀분석을 수행하시오. 

통계적 유의성을 갖지 못하는 독립변수를 제거하면 회귀 모형에 남는 변수는 모두 몇 개인가? 

이 때 음의 회귀계수를 가지는 변수는 몇 개인가? 

Python 함수 가이드
```
from sklearn.linear_model import LinearRegression 또는
from statsmodels.formula.api import ols 중 적합한 것을 사용하세요
```

In [64]:
from statsmodels.formula.api import ols

formula = 'price ~ ' + '+'.join([i for i in df_06.columns if i not in ['id', 'date', 'zipcode', 'price']])
lm = ols(formula, data=df_06).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.681
Model:                            OLS   Adj. R-squared:                  0.680
Method:                 Least Squares   F-statistic:                     656.0
Date:                Wed, 26 Jul 2023   Prob (F-statistic):               0.00
Time:                        22:06:28   Log-Likelihood:                -58960.
No. Observations:                4323   AIC:                         1.180e+05
Df Residuals:                    4308   BIC:                         1.180e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      6.302e+06   2.95e+05     21.401      0.000    5.73e+06    6.88e+06
bedrooms        -2.5e+04   3680.510     -6.791      0.000   -3.22e+04   -1.78e+04
bathrooms      5.179e+04   7163.408      7.230      0.000    3.77e+04    6.58e+04
sqft_living      92.3080      5.074     18.192      0.000      82.360     102.256
sqft_lot          0.0054      0.093      0.059      0.953      -0.176       0.187
floors         3.748e+04   8141.392      4.604      0.000    2.15e+04    5.34e+04
waterfront     6.277e+05   3.91e+04     16.064      0.000    5.51e+05    7.04e+05
view            4.04e+04   4797.344      8.421      0.000     3.1e+04    4.98e+04
condition      1.571e+04   5317.387      2.954      0.003    5280.918    2.61e+04
grade          1.306e+05   4700.604     27.789      0.000    1.21e+05     1.4e+05
sqft_above       33.7358      4.981      6.773      0.000      23.971      43.501
sqft_basement    58.5722      5.840     10.030      0.000      47.123      70.021
yr_built      -3666.8957    150.805    -24.316      0.000   -3962.551   -3371.241
yr_renovated     17.7032      8.265      2.142      0.032       1.499      33.908
sqft_living15    23.0114      7.561      3.044      0.002       8.188      37.834
sqft_lot15       -0.2323      0.142     -1.637      0.102      -0.511       0.046
==============================================================================
Omnibus:                     2170.721   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            38362.875
Skew:                           1.971   Prob(JB):                         0.00
Kurtosis:                      17.051   Cond. No.                     1.37e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 8.59e-22. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

유의성 없음: sqft_living, sqft_lot15	

In [50]:
len([i for i in df_06.columns if i not in ['id', 'date', 'zipcode', 'price']]) - 2

13

**유의성 중 음의 상관계수: 2개**

**13, 2**